# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_to_load = "../WeatherPy/Output/weather_data.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data = weather_data.drop(["ID"],axis=1)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,tasiilaq,75,GL,1585931806,53,65.61,-37.64,24.80,17.22
1,honningsvag,95,NO,1585931879,79,70.98,25.97,21.20,12.75
2,hobart,90,AU,1585931879,77,-42.88,147.33,59.00,5.82
3,half moon bay,1,US,1585931879,62,37.46,-122.43,54.00,17.22
4,levanger,75,NO,1585931880,89,63.75,11.30,32.00,25.28
...,...,...,...,...,...,...,...,...,...
550,pitiquito,43,MX,1585932009,35,30.70,-112.08,78.01,3.00
551,datong,0,CN,1585932009,32,40.09,113.29,44.22,4.43
552,vilhena,75,BR,1585931896,74,-12.74,-60.15,84.20,5.82
553,muyezerskiy,89,RU,1585932009,95,63.93,31.65,26.67,4.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data.Humidity

In [5]:
humidity = humidity.astype(float)

In [6]:
fig = gmaps.figure(zoom_level=3,center=(30.0902, -30.7129),layout={'width': '800px', 'height': '800px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='800px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_df = weather_data[(weather_data["Max Temp"] < 80) &
                        (weather_data["Max Temp"] > 70) &
                       (weather_data["Wind Speed"] < 10) &
                       (weather_data.Cloudiness == 0)]

ideal_df = ideal_df.dropna()
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
52,trincomalee,0,LK,1585931891,79,8.57,81.23,79.09,6.82
77,castro,0,BR,1585931705,26,-24.79,-50.01,70.92,6.51
158,shibganj,0,BD,1585931919,16,24.68,88.16,79.77,4.90
177,gaya,0,IN,1585931923,46,24.78,85.00,73.40,3.20
187,pisco,0,PE,1585931925,30,-13.70,-76.22,75.00,8.05
270,kruisfontein,0,ZA,1585931945,57,-34.00,24.73,71.42,6.15
335,plettenberg bay,0,ZA,1585931740,57,-34.05,23.37,72.88,4.85
382,venado tuerto,0,AR,1585931770,41,-33.75,-61.97,72.00,6.87
390,didwana,0,IN,1585931971,16,27.40,74.57,77.49,7.14
398,batticaloa,0,LK,1585931973,79,7.71,81.69,79.56,4.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [9]:
for index, row in hotel_df.iterrows():
    hotel_lat = str(row["Lat"])
    hotel_lng = str(row["Lng"])
    hotel_coordinates = hotel_lat + ", " + hotel_lng
    
    params = {
        "location": hotel_coordinates,
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    try:
        hotel_name = (response.json()["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except:
        print(f"Unable to find hotel for {row.City}, {row.Country}.")

Unable to find hotel for shibganj, BD.
Unable to find hotel for kruisfontein, ZA.


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='800px', width='800px'))